In [ ]:
import numpy as np
import sys
from suite2p import run_s2p

# populates ops with the default options
ops = run_s2p.default_ops()
# specify a folder with sbx files
ops['data_path'] = ['e:/data/2photon/raw/191103_JC121_2P_V1_JC/run02_speedup_8dir_whiskers']

In [ ]:
# update options from a scanbox file (multi-plane experiment; supports only sawtooth mode)
from suite2p.io.utils import search_for_ext
from suite2p.io.sbx import sbx_get_info
# get parameters from the first sbx file
fname = search_for_ext(ops['data_path'][0],'sbx')
info = sbx_get_info(fname[0])
# number of planes
nplanes = 1
if info.volscan and not isinstance(info.otwave,int):
    nplanes = len(info.otwave)
# number of channels
nchannels = 1
if info.channels == 1:
    nchannels = 2
# aspect ratio and mag
magidx = info.config.magnification - 1
um_per_pix_x = info.calibration[magidx].x
um_per_pix_y = info.calibration[magidx].y
mag = info.config.magnification_list[magidx]
# frame rate
factor = 2 if info.scanmode == 0 else 1
fs = factor*(info.resfreq/info.config.lines)/nplanes
ops['input_format'] = 'sbx'
if info.scanmode == 1:
    # do not remove dead columns
    ops['sbx_ndeadcols'] = 0
else:
    # remove dead columns when in bidi mode(trim data)
    ops['sbx_ndeadcols'] = -1
# set suite2p ops
ops['fs'] = fs/nplanes
ops['nplanes'] = nplanes
ops['nchannels'] = nchannels
ops['aspect'] = um_per_pix_x/um_per_pix_y
# add some extra info to the ops
ops['magnification'] = mag
ops['um_per_pixel_x'] = um_per_pix_x
ops['um_per_pixel_y'] = um_per_pix_y

# run one experiment
opsEnd=run_s2p(ops=ops,db={})